In [1]:
from pybullet_environment import *
from guide import *

import numpy as np
import openGJK_cython as opengjk

import torch
import transformations

pybullet build time: Jul 17 2023 13:38:55


In [2]:
env = RobotEnvironment(manipulator = True)
guide = IntersectionVolumeGuide(env)

argv[0]=
startThreads creating 1 threads.
starting thread 0
started thread 0 
argc=3
argv[0] = --unused
argv[1] = 
argv[2] = --start_demo_name=Physics Server
ExampleBrowserThreadFunc started
X11 functions dynamically loaded using dlopen/dlsym OK!
X11 functions dynamically loaded using dlopen/dlsym OK!
Creating context
Created GL 3.3 context
Direct GLX rendering context obtained
Making context current
GL_VENDOR=NVIDIA Corporation
GL_RENDERER=NVIDIA GeForce GTX 1660 Ti/PCIe/SSE2
GL_VERSION=3.3.0 NVIDIA 535.86.05
GL_SHADING_LANGUAGE_VERSION=3.30 NVIDIA via Cg compiler
pthread_getconcurrency()=0
Version = 3.3.0 NVIDIA 535.86.05
Vendor = NVIDIA Corporation
Renderer = NVIDIA GeForce GTX 1660 Ti/PCIe/SSE2
b3Printf: Selected demo: Physics Server
startThreads creating 1 threads.
starting thread 0
started thread 0 
MotionThreadFunc thread started


: 

In [3]:
joint_id = 3
link_id = 2

joint_config = np.array([-1.9300085,  1.3141115,  2.1577177, -1.5229155, -0.7427585, 1.4192199, -1.3984755])
for i, joint_ind in enumerate(env.joints):
    env.client_id.resetJointState(env.manipulator, joint_ind, joint_config[i])

fk = (guide.forward_kinematics(joint_id, torch.tensor(joint_config))).numpy()
fk_inv = np.linalg.inv(fk)

frame_pos, frame_ori = env.client_id.getLinkState(env.manipulator, link_id)[4:6]
link_transform = env.pose_to_transformation(np.array([*frame_pos, *frame_ori]))

x1 = link_transform @ fk_inv

joint_config = np.array([-1.619, 1.51, 1.425, -1.58850, -0.9250, 1.52913, -0.899883])
for i, joint_ind in enumerate(env.joints):
    env.client_id.resetJointState(env.manipulator, joint_ind, joint_config[i])

fk = (guide.forward_kinematics(joint_id, torch.tensor(joint_config))).numpy()
fk_inv = np.linalg.inv(fk)

env.draw_frame(fk)

frame_pos, frame_ori = env.client_id.getLinkState(env.manipulator, link_id)[4:6]
link_transform = env.pose_to_transformation(np.array([*frame_pos, *frame_ori]))

x2 = link_transform @ fk_inv

print(x2 - x1)

# print(env.link_poses[0][:3])
# print(np.array(env.client_id.getEulerFromQuaternion(env.link_poses[0][3:])) * 180 / np.pi)
# link_transform = env.pose_to_transformation(env.link_poses[link_id])

[[-0.51260864  0.00519596 -0.02712849  0.00903378]
 [ 0.28385966  0.77498348 -0.43257279  0.30225519]
 [ 0.74170311 -0.26848391  0.18941703 -0.17430876]
 [ 0.          0.          0.          0.        ]]


: 

In [9]:
id = 1
link_transform = env.pose_to_transformation(env.link_poses[id])
fk = (guide.forward_kinematics(id, torch.tensor(np.zeros((7,))))).numpy()
fk_inv = np.linalg.inv(fk)

env.draw_frame(link_transform)

print(fk)
print(link_transform)

x = link_transform @ fk_inv
print(x)

[[1.    0.    0.    0.   ]
 [0.    1.    0.    0.   ]
 [0.    0.    1.    0.333]
 [0.    0.    0.    1.   ]]
[[ 1.00000000e+00 -0.00000000e+00  0.00000000e+00 -8.42500000e-05]
 [ 0.00000000e+00  0.00000000e+00  1.00000000e+00  3.71962000e-02]
 [-0.00000000e+00 -1.00000000e+00  0.00000000e+00  4.02395004e-01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
[[ 1.00000000e+00  0.00000000e+00  0.00000000e+00 -8.42500000e-05]
 [ 0.00000000e+00  0.00000000e+00  1.00000000e+00 -2.95803804e-01]
 [ 0.00000000e+00 -1.00000000e+00  0.00000000e+00  4.02395004e-01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]


In [11]:
env.draw_frame(fk)

X connection to :1 broken (explicit kill or server shutdown).


: 

In [ ]:
# Joint 0 to Link 0:
[[ 1.00000000e+00  0.00000000e+00  0.00000000e+00  8.71000000e-05]
 [ 0.00000000e+00  1.00000000e+00  0.00000000e+00 -3.70903500e-02]
 [ 0.00000000e+00  0.00000000e+00  1.00000000e+00  2.64484554e-01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]

# Joint 1 to Link 1:

In [5]:
fk = (guide.forward_kinematics(1, torch.tensor(np.zeros((7,))))).numpy()
print(fk)

[[1.    0.    0.    0.   ]
 [0.    1.    0.    0.   ]
 [0.    0.    1.    0.333]
 [0.    0.    0.    1.   ]]


In [4]:
for i in range(1, 7):

    fk = (guide.forward_kinematics(i, torch.tensor(joint_config))).numpy()
    env.draw_frame(fk)
    _ = input("")

: 

In [9]:

link_transform = env.pose_to_transformation(env.link_poses[3])
env.draw_frame(link_transform, scale_factor = 0.5)
    #_ = input("")

: 

In [14]:
def get_unit_axes_from_transform(transform_matrix):
    """
    Extract unit X, Y, and Z axes of the world frame in the transformed frame.
    
    :param transform_matrix: 4x4 transformation matrix
    :return: Array of size (2x3) representing the unit X, Y, and Z axes in the transformed frame
    """
    rotation_matrix = transform_matrix[:3, :3]
    unit_axes_world = np.array([[1, 0, 0], [0, 1, 0], [0, 0, 1]])
    unit_axes_transformed = np.dot(rotation_matrix, unit_axes_world.T).T
    
    return unit_axes_transformed[:2]

# Example 4x4 transformation matrix
transform_matrix = np.array([
    [0.7071, -0.7071, 0.0, 1.0],
    [0.7071, 0.7071, 0.0, 2.0],
    [0.0, 0.0, 1.0, 3.0],
    [0.0, 0.0, 0.0, 1.0]
])

unit_axes = get_unit_axes_from_transform(transform_matrix)
print("Unit Axes:")
print(unit_axes)

Unit Axes:
[[ 0.7071  0.7071  0.    ]
 [-0.7071  0.7071  0.    ]]


: 

In [28]:
print(env.client_id.getBasePositionAndOrientation(env.manipulator))
print(env.client_id.getLinkState(env.manipulator, 0)[4:6])

((0.0, 0.0, 0.05), (0.0, 0.0, 0.0, 1.0))
((0.0, 0.0, 0.3330000042915344), (0.0, 0.0, 0.0, 1.0))


In [ ]:
link_displacements = [[-0.03475364, 0.01295708, -0.06851545],
                      [0.01123399  -0.07589314 0.01769917],
                      [0.03306312 0.03418906 0.03651213],
                      [0.02470827  -0.05497029 0.00575065],
                      [-0.08881844 0.05518941  -0.04171566],
                      [0.03112165  -0.01785765 0.02759352],
                      [0.05384276 0.04399419  -0.04652401]]

[[ 0.76656756 -0.50092286 -0.40180881  0.18912465]
 [ 0.61213413  0.75910694  0.22146881 -0.41518173]
 [ 0.19407718 -0.41573173  0.88853875 -0.27538774]
 [ 0.          0.          0.          1.        ]]

[[ 0.76656756 -0.50092286 -0.40180881  0.22882068]
 [ 0.61213413  0.75910694  0.22146881 -0.38167038]
 [ 0.19407718 -0.41573173  0.88853875 -0.31778107]
 [ 0.          0.          0.          1.        ]]

In [3]:
env.draw_link_bounding_boxes()

In [53]:
print(env.link_bounding_objs)
print(len(env.link_poses))

[11, 10, 9, 8, 7, 6, 5, 4, 3, 2]
10


In [30]:
hand_pose, hand_ori = env.client_id.getBasePositionAndOrientation(10)

change = env.client_id.getQuaternionFromEuler([0., 0., np.pi/2])
original = env.client_id.getEulerFromQuaternion(hand_ori)

np.array(original) * 180 / np.pi

array([180.,  -0.,   0.])

In [10]:
env.clear_bounding_boxes()

In [54]:
    
env.client_id.removeBody(11)

: 

## Start Joint Configuration:

#### From End-effector pose

In [35]:
start = np.array([0.5,  -0.3,  0.1])
start_orientation = np.array([-3, -0.6, 1.57])

joint_config = env.inverse_kinematics(start, start_orientation)[:-2]
print(joint_config)

for i, joint_ind in enumerate(env.joints):
    env.client_id.resetJointState(env.manipulator, joint_ind, joint_config[i])

(-1.619321477803221, 1.5097250391784747, 1.42505411264574, -1.5885010292282844, -0.9250272674133665, 1.5291345230583324, -0.8998830755305524)


#### From Joint angles

In [12]:
joint_config = np.array([-1.9300085,  1.3141115,  2.1577177, -1.5229155, -0.7427585, 1.4192199, -1.3984755])

for i, joint_ind in enumerate(env.joints):
    env.client_id.resetJointState(env.manipulator, joint_ind, joint_config[i])

In [7]:
joint_config = np.array([-1.619, 1.51, 1.425, -1.58850, -0.9250, 1.52913, -0.899883])

for i, joint_ind in enumerate(env.joints):
    env.client_id.resetJointState(env.manipulator, joint_ind, joint_config[i])

## End Joint Configuration:

#### From End-effector pose

In [3]:
goal = np.array([0.5,  0.01,  0.4])
goal_orientation = np.array([3.14, 0, 0])

joint_config = env.inverse_kinematics(goal, goal_orientation)[:-2]
print(joint_config)

for i, joint_ind in enumerate(env.joints):
    env.client_id.resetJointState(env.manipulator, joint_ind, joint_config[i])

(0.022802318029354115, 1.2834343703383035, 0.019562611797868577, 0.5781106044689629, -0.03151996619547765, 0.6832525013982176, 0.7888675395724642)


#### From Joint angles

In [16]:
joint_config = np.zeros((9,))[:-2]

for i, joint_ind in enumerate(env.joints):
    env.client_id.resetJointState(env.manipulator, joint_ind, joint_config[i])

# -------------------------------------------------------------------------------

## Test Obstacle Collision:

#### Define Obstacle A:

In [3]:
A_pos = np.array([0.2, 0.1, 0.5])
A_ori = np.array([0., 0., 0.])
A_quat = np.array(env.client_id.getQuaternionFromEuler(A_ori))
A_pose = np.array([*A_pos, *A_quat])

A_size = np.array([0.5, 0.3, 0.2])

vuid = env.client_id.createVisualShape(p.GEOM_BOX, 
                                       halfExtents = A_size/2,
                                       rgbaColor = np.hstack([env.colors['yellow'], np.array([1.0])]))

A_id = env.client_id.createMultiBody(baseVisualShapeIndex = vuid, 
                                       basePosition = A_pos, 
                                       baseOrientation = A_quat)

In [4]:
B_pos = np.array([0.21, 0.11, 0.51])
B_ori = np.array([0.1, 0.3, -0.4])
B_quat = np.array(env.client_id.getQuaternionFromEuler(B_ori))
B_pose = np.array([*B_pos, *B_quat])

B_size = np.array([0.15, 0.3, 0.08])

vuid = env.client_id.createVisualShape(p.GEOM_BOX, 
                                       halfExtents = B_size/2,
                                       rgbaColor = np.hstack([env.colors['red'], np.array([1.0])]))

B_id = env.client_id.createMultiBody(baseVisualShapeIndex = vuid, 
                                       basePosition = B_pos, 
                                       baseOrientation = B_quat)

In [7]:
x = torch.tensor(B_pose, dtype = torch.float32)
x.requires_grad = True

a = torch.tensor(A_pose)
x_size = torch.tensor(B_size)
a_size = torch.tensor(A_size)

a_rot = quaternion_to_rotation_matrix(a[3:])

x_vertices_static = torch.tensor([[-x_size[0]/2, -x_size[1]/2, -x_size[2]/2],
                       [ x_size[0]/2, -x_size[1]/2, -x_size[2]/2],
                       [ x_size[0]/2,  x_size[1]/2, -x_size[2]/2],
                       [-x_size[0]/2,  x_size[1]/2, -x_size[2]/2],
                       [-x_size[0]/2, -x_size[1]/2,  x_size[2]/2],
                       [ x_size[0]/2, -x_size[1]/2,  x_size[2]/2],
                       [ x_size[0]/2,  x_size[1]/2,  x_size[2]/2],
                       [-x_size[0]/2,  x_size[1]/2,  x_size[2]/2]], dtype = torch.float32).T

a_vertices_static = torch.tensor([[-a_size[0]/2, -a_size[1]/2, -a_size[2]/2],
                       [ a_size[0]/2, -a_size[1]/2, -a_size[2]/2],
                       [ a_size[0]/2,  a_size[1]/2, -a_size[2]/2],
                       [-a_size[0]/2,  a_size[1]/2, -a_size[2]/2],
                       [-a_size[0]/2, -a_size[1]/2,  a_size[2]/2],
                       [ a_size[0]/2, -a_size[1]/2,  a_size[2]/2],
                       [ a_size[0]/2,  a_size[1]/2,  a_size[2]/2],
                       [-a_size[0]/2,  a_size[1]/2,  a_size[2]/2]], dtype = torch.float32).T

a_vertices = (a_rot @ a_vertices_static) + a[:3].unsqueeze(1)

In [8]:
for _ in range(2000):

    x.grad = None

    x_rot = quaternion_to_rotation_matrix(x[3:])

    x_vertices = (x_rot @ x_vertices_static) + x[:3].unsqueeze(1)

    x_min = torch.min(x_vertices, dim = 1)[0]
    x_max = torch.max(x_vertices, dim = 1)[0]

    a_min = torch.min(a_vertices, dim = 1)[0]
    a_max = torch.max(a_vertices, dim = 1)[0]

    overlap_min = torch.max(x_min, a_min)
    overlap_max = torch.min(x_max, a_max)

    overlap_lengths = overlap_max - overlap_min
    volume = torch.prod(torch.clamp(overlap_lengths, min = 0))

    volume.backward()

    with torch.no_grad():
        x -= 0.01 * x.grad

    env.client_id.resetBasePositionAndOrientation(B_id, x[:3].detach().numpy(), x[3:].detach().numpy())

: 

In [78]:
volume.backward()
print(x.grad)

tensor([-0.0342,  0.0345, -0.0443,  0.0111,  0.0056, -0.0089, -0.0036])


In [6]:
env.client_id.removeBody(B_id)

# -------------------------------------------------------------------------------

In [6]:
def quaternion_to_rotation_matrix(quaternion):
    # Ensure the quaternion is normalized
    quaternion = quaternion / torch.norm(quaternion)
    
    w, x, y, z = quaternion[0], quaternion[1], quaternion[2], quaternion[3]
    rotation_matrix = torch.zeros((3, 3), dtype=torch.float32)
    
    rotation_matrix[0, 0] = 1 - 2 * (y * y + z * z)
    rotation_matrix[0, 1] = 2 * (x * y - w * z)
    rotation_matrix[0, 2] = 2 * (x * z + w * y)
    
    rotation_matrix[1, 0] = 2 * (x * y + w * z)
    rotation_matrix[1, 1] = 1 - 2 * (x * x + z * z)
    rotation_matrix[1, 2] = 2 * (y * z - w * x)
    
    rotation_matrix[2, 0] = 2 * (x * z - w * y)
    rotation_matrix[2, 1] = 2 * (y * z + w * x)
    rotation_matrix[2, 2] = 1 - 2 * (x * x + y * y)
    
    return rotation_matrix

In [18]:
start = [-1.6055961453458505, 1.2177152063074654, 1.2116691324250535, -1.7072048998181064, -0.6481964392335137, 1.7371698235121535, 
         -1.0210010166146453]

for i, joint_ind in enumerate(env.joints):
    env.client_id.resetJointState(env.manipulator, joint_ind, start[i])


In [4]:
joint_config = np.array([-0.7856447,  0.7866443,  2.689896 , -1.9357901, -0.3345264,
        1.2200228, -2.0265737])
for i, joint_ind in enumerate(env.joints):
    env.client_id.resetJointState(env.manipulator, joint_ind, joint_config[i])

In [6]:
env.client_id.disconnect()

numActiveThreads = 0
stopping threads
Thread with taskId 0 exiting
Thread TERMINATED
destroy semaphore
semaphore destroyed
destroy main semaphore
main semaphore destroyed
finished
numActiveThreads = 0
btShutDownExampleBrowser stopping threads
Thread with taskId 0 exiting
Thread TERMINATED
destroy semaphore
semaphore destroyed
destroy main semaphore
main semaphore destroyed
